# 개선된 RAG Chain 생성

특정 문자열 검색이 잘 되도록 개선된 텍스트 분할 및 임베딩 방식

In [111]:
import os
import re
from typing import List, Dict, Any
from pathlib import Path
from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import json

In [98]:
def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드
md_content = load_markdown_file("data/dev_center_guide_touched.md")
print(f"마크다운 파일 크기: {len(md_content)} 문자")
print(f"첫 500자: {md_content[:500]}...")

마크다운 파일 크기: 112832 문자
첫 500자: # 원스토어 인앱 결제 연동 가이드

## 원스토어 In-App SDK
원스토어 인앱결제는 다양한 결제수단과 강력한 보안 그리고 편리하고 안전한 결제 서비스를 제공합니다.

- 최신버전 인앱결제 가이드
[원스토어 인앱결제 API V7 (SDK V21) 안내 및 다운로드](#원스토어-인앱결제-api-v7sdk-v21-연동-안내-및-다운로드)

## 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

- API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 여기를 클릭해주세요.
- 현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요
  - [대한민국 외 국가 및 지역 배포를 위한 가이드](https://onestore-de...


In [113]:
def improved_markdown_split(md_text: str, chunk_size: int = 300, chunk_overlap: int = 50) -> List[Document]:
    """
    개선된 마크다운 분할 함수
    - 더 작은 청크 크기로 분할
    - 코드 블록과 특수문자 보존
    - 키워드 기반 청크 생성
    """
    
    # 1단계: 헤더 기반 분할
    header_pattern = r'^(#{1,6})\s+(.+)$'
    headers = re.findall(header_pattern, md_text, re.MULTILINE)
    
    # 2단계: 코드 블록 보존하면서 분할
    # 코드 블록을 임시로 마스킹
    code_blocks = {}
    counter = 0
    
    def mask_code_block(match):
        nonlocal counter
        placeholder = f"__CODE_BLOCK_{counter}__"
        code_blocks[placeholder] = match.group(0)
        counter += 1
        return placeholder
    
    # 코드 블록 마스킹
    masked_text = re.sub(r'```[\\s\\S]*?```', mask_code_block, md_text)
    
    # 3단계: 개선된 텍스트 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[
            "\\n\\n",  # 빈 줄
            "\\n",    # 줄바꿈
            ". ",    # 문장 끝
            ", ",    # 쉼표
            " ",     # 공백
            ""        # 문자 단위
        ],
        length_function=len,
        is_separator_regex=False
    )
    
    chunks = splitter.split_text(masked_text)
    
    # 4단계: 코드 블록 복원 및 문서 생성
    documents = []
    
    for i, chunk in enumerate(chunks):
        # 코드 블록 복원
        restored_chunk = chunk
        for placeholder, code_block in code_blocks.items():
            if placeholder in chunk:
                restored_chunk = restored_chunk.replace(placeholder, code_block)
        
        # 헤더 정보 추출
        header_info = ""
        lines = restored_chunk.split('\\n')
        for line in lines:
            if line.strip().startswith('#'):
                header_info = line.strip()
                break
        
        # 키워드 추출 (함수명, API명 등)
        keywords = extract_keywords(restored_chunk)
        
        doc = Document(
            page_content=restored_chunk,
            metadata={
                "type": "documentation",
                "source": "dev_center_guide_touched.md",
                "chunk_idx": i,
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap,
                "header": header_info,
                "keywords": keywords,
                "has_code": "```" in restored_chunk,
                "has_function": any(func in restored_chunk for func in ["()", "function", "API", "launch"])
            }
        )
        documents.append(doc)
    
    return documents

def extract_keywords(text: str) -> List[str]:
    """텍스트에서 키워드를 추출합니다."""
    keywords = []
    
    # 함수명 패턴 (예: launchPurchaseFlow, queryProductDetailAsync)
    function_pattern = r'\\b[a-zA-Z][a-zA-Z0-9]*\\([^)]*\\)'
    functions = re.findall(function_pattern, text)
    keywords.extend(functions)
    
    # API 패턴
    api_pattern = r'\\b[A-Z][A-Z0-9_]*\\b'
    apis = re.findall(api_pattern, text)
    keywords.extend(apis[:5])  # 상위 5개만
    
    # 특수 키워드
    special_keywords = [
        "launchPurchaseFlow", "queryProductDetailAsync", "PurchaseClient",
        "IapResult", "PurchaseData", "ProductDetail", "PurchaseFlowParams", "PNS"
    ]
    
    for keyword in special_keywords:
        if keyword.lower() in text.lower():
            keywords.append(keyword)
    
    return list(set(keywords))  # 중복 제거

# 개선된 분할 실행
docs_improved = improved_markdown_split(md_content, chunk_size=300, chunk_overlap=50)
print(f"개선된 분할 완료: {len(docs_improved)}개 청크")

# launchPurchaseFlow 포함 문서 확인
launch_docs = []
for i, doc in enumerate(docs_improved):
    if 'PNS' in doc.page_content:
        launch_docs.append((i, doc))
        print(f"문서 {i}: {doc.page_content}...")
        print(f"키워드: {doc.metadata.get('keywords', [])}")
        print("-" * 50)

print(f"\\n총 {len(launch_docs)}개의 문서에서 'PNS' 발견")

개선된 분할 완료: 548개 청크
문서 358: . PNS (Push Notification Service) 이용하기
#### 개요 
원스토어는 개발자를 위해 두 가지 Push Notification Service를 제공합니다.
* PNS는 Push Notification Service의 약자입니다.
* PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 개별 사용자의 결제 상태(결제 완료...
키워드: ['PNS']
--------------------------------------------------
문서 360: 전송됩니다.

- 인앱상품 결제 또는 결제취소가 발생하면 원스토어가 개발사 서버로 알림을 전송하는 PNS(Payment Notification Service) 

- 구독 상태가 변경되면 개발사 서버로 알림을 전송하는 SNS (Subscription Notifacation Service) 

> Notification은 발송/수신 서버의 상태에 따라 지연 또는 유실될 수 있으므로...
키워드: ['PNS']
--------------------------------------------------
문서 362: . 
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.
원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하게 notification이 발송됩니다...
키워드: ['PNS']
--------------------------------------------------
문서 363: . 원스토어가 진행한 결제 테스트 내역은 주기적으로 원스토어에서 자체 취소 처리합니다.

#### PNS 수신 서버 URL 설정
PNS를 수신 받을 개발사 서버의 URL은 '개발자센터 > Apps > 상품 선택 > In-App정보' 메뉴에서 'PNS 관

In [101]:
def create_enhanced_embeddings(docs: List[Document], output_path: str):
    """향상된 임베딩 생성 및 저장"""
    
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
    
    # FAISS 데이터베이스 생성
    db = FAISS.from_documents(docs, embedding_model)
    
    # 추가 인덱스 생성 (키워드 기반)
    keyword_docs = []
    for doc in docs:
        keywords = doc.metadata.get('keywords', [])
        for keyword in keywords:
            # 키워드별로 별도 문서 생성
            keyword_doc = Document(
                page_content=f"{keyword}: {doc.page_content[:200]}...",
                metadata={
                    **doc.metadata,
                    "keyword": keyword,
                    "is_keyword_doc": True
                }
            )
            keyword_docs.append(keyword_doc)
    
    # 키워드 문서도 임베딩에 추가
    if keyword_docs:
        keyword_db = FAISS.from_documents(keyword_docs, embedding_model)
        # 기존 DB와 병합
        db.merge_from(keyword_db)
    
    # 저장
    db.save_local(output_path)
    print(f"✅ 향상된 임베딩 저장 완료: {output_path}")
    print(f"총 문서 수: {len(docs) + len(keyword_docs)}")
    
    return db

# 향상된 임베딩 생성
output_dir = "models/faiss_vs_rag_iap_v5_enhanced"
os.makedirs(output_dir, exist_ok=True)
enhanced_db = create_enhanced_embeddings(docs_improved, output_dir)

✅ 향상된 임베딩 저장 완료: models/faiss_vs_rag_iap_v5_enhanced
총 문서 수: 729


In [110]:
# 향상된 검색 테스트
print("=== 향상된 검색 테스트 ===")

# 임베딩 로드
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

# 다양한 검색 방식 테스트
test_queries = [
    # "launchPurchaseFlow()",
    # "launchPurchaseFlow",
    # "구매 요청 함수",
    # "PurchaseClient",
    # "queryProductDetailAsync",
    # "인앱결제 구매 프로세스",
    # "IapResult",
    # "PurchaseData",
    "PNS"
]

for query in test_queries:
    print(f"\\n🔍 검색 쿼리: {query}")
    
    # 1. 직접 similarity 검색
    results_direct = loaded_db.similarity_search(query, k=5)
    print(f"\\n1. 직접 검색 결과 ({len(results_direct)}개):")
    for i, doc in enumerate(results_direct, 1):
        print(f"  {i}. {doc.page_content}")
        print(f"     키워드: {doc.metadata.get('keywords', [])}")
        print("-" * 80)
    
    # 2. MMR 검색
    retriever_mmr = loaded_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.6}
    )
    results_mmr = retriever_mmr.invoke(query)
    print("=" * 80)
    print(f"\\n2. MMR 검색 결과 ({len(results_mmr)}개):")
    for i, doc in enumerate(results_mmr, 1):
        print(f"  {i}. {doc.page_content}")
        print(f"     키워드: {doc.metadata.get('keywords', [])}")
        print("-" * 80)
    
    print("=" * 80)

=== 향상된 검색 테스트 ===
\n🔍 검색 쿼리: PNS
\n1. 직접 검색 결과 (5개):
  1. . "
     키워드: []
--------------------------------------------------------------------------------
  2. . 다만
     키워드: []
--------------------------------------------------------------------------------
  3. . 단
     키워드: []
--------------------------------------------------------------------------------
  4. PurchaseClient: . 
        // PurchaseClient by calling the startConnection() method.
    }
}
```

### 05. 05...
     키워드: ['PurchaseClient']
--------------------------------------------------------------------------------
  5. , please change the language to English from the upper left side in this page.

### 01
     키워드: []
--------------------------------------------------------------------------------
\n2. MMR 검색 결과 (5개):
  1. . "
     키워드: []
--------------------------------------------------------------------------------
  2. PurchaseClient: . 
        // PurchaseClient by calling the startConnection() method.
    }
}

In [102]:
# 향상된 RAG 체인 생성
print("=== 향상된 RAG 체인 생성 ===")

# LLM 설정
llm = ChatOllama(model="exaone3.5:latest", temperature=0.3)

# 검색기 설정 (하이브리드 검색)
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 15, "lambda_mult": 0.6}
)

# 향상된 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요
6. 함수명이나 API명이 언급된 경우 구체적인 사용법을 설명해주세요

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ 향상된 RAG 체인이 생성되었습니다!")

=== 향상된 RAG 체인 생성 ===
✅ 향상된 RAG 체인이 생성되었습니다!


In [106]:
# RAG 체인 테스트
print("=== RAG 체인 테스트 ===")

test_questions = [
    "launchPurchaseFlow() 함수는 언제 사용되나요?",
    # "PurchaseClient 초기화 방법을 알려주세요",
    # "queryProductDetailAsync API 사용법을 설명해주세요",
    # "IapResult와 PurchaseData의 차이점은?",
    # "인앱결제 구매 프로세스를 단계별로 설명해주세요"
    "PNS(Push Notification Service)는 무엇인가요?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

=== RAG 체인 테스트 ===
\n📝 질문 1: launchPurchaseFlow() 함수는 언제 사용되나요?
------------------------------------------------------------
`launchPurchaseFlow()` 함수는 원스토어 인앱 결제 시스템에서 사용자가 인앱 상품을 구매할 때 결제 프로세스를 시작하는 데 사용됩니다. 구체적으로 다음과 같은 상황에서 활용됩니다:

### 사용 시기
1. **인앱 상품 구매 시작**: 사용자가 인앱 상품을 구매하려고 할 때, 결제 프로세스를 시작하기 위해 이 함수를 호출합니다.
2. **결제 요청**: 결제 요청을 서버나 결제 게이트웨이로 보내는 시점에서 사용됩니다. 이 함수는 사용자의 결제 정보를 처리하고 결제를 완료하기 위한 요청을 생성합니다.

### 단계별 설명
1. **함수 호출**: `launchPurchaseFlow()` 함수를 호출하며, 이때 필요한 파라미터를 전달합니다. 주로 `PurchaseFlowParams` 객체를 사용합니다.
   
   ```kotlin
   fun launchPurchaseFlow(purchaseParam: PurchaseFlowParams) {
       executeServiceRequest {
           // 결제 요청 처리 로직
           Log.i(TAG, "==================================================")
           Log.i(TAG, "productId token : ${purchaseParam.productId}")
           // 결제 결과 처리 로직
       }
   }
   ```

2. **파라미터 전달**: `PurchaseFlowParams` 객체에는 구매하려는 상품의 ID(`productId`), 결제 토큰(`token`), 그리고 기타 필요한 결제 관련 정보가 포함됩니다.
   
   ```kotlin
   data c

In [107]:
# 대화형 질문 응답
def ask_question(question: str) -> str:
    """질문에 대한 답변을 생성합니다."""
    try:
        answer = rag_chain.invoke(question)
        return answer
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

print("💬 대화형 질문 응답 테스트\\n")

# 샘플 질문들
sample_questions = [
    "launchPurchaseFlow 함수의 매개변수는 무엇인가요?",
    "결제 실패 시 어떻게 처리하나요?",
    "정기결제와 일회성 결제의 차이점은?",
    "PurchaseFlowParams.Builder 사용법을 알려주세요"
]

for question in sample_questions:
    print(f"\\n❓ 질문: {question}")
    print("🔍 검색 중...")
    
    answer = ask_question(question)
    print(f"\\n🤖 답변:\\n{answer}")
    print("-" * 60)

💬 대화형 질문 응답 테스트\n
\n❓ 질문: launchPurchaseFlow 함수의 매개변수는 무엇인가요?
🔍 검색 중...
\n🤖 답변:\n주어진 컨텍스트에는 `launchPurchaseFlow` 함수에 대한 직접적인 정보가 포함되어 있지 않습니다. 원스토어 인앱 결제 SDK V7(SDK V21) 관련 문서와 코드 예시들은 주로 다른 함수와 클래스들에 초점을 맞추고 있습니다. 따라서 `launchPurchaseFlow` 함수의 정확한 매개변수에 대한 정보를 제공하기는 어렵습니다.

그러나 일반적으로 원스토어 인앱 결제 SDK에서 구매 프로세스를 시작하는 함수의 매개변수는 다음과 같은 종류를 포함할 가능성이 높습니다:

1. **Context**: 액티비티 컨텍스트를 제공하여 인앱 결제 프로세스를 실행할 수 있도록 합니다.
2. **String**: 구매 아이템의 고유 식별자 (Item ID)를 포함합니다.
3. **PurchaseListener 또는 Callback 객체**: 구매 결과를 처리하기 위한 콜백 함수나 객체입니다.
4. **Bundle**: 추가적인 구매 옵션이나 매개변수를 전달하기 위한 키-값 쌍입니다.

**예시 코드 (가상 예시)**:

```kotlin
fun launchPurchaseFlow(context: Context, itemId: String, listener: PurchaseListener) {
    // 구매 프로세스를 시작하는 가상 함수 예시
    val bundle = Bundle()
    // 추가 옵션을 Bundle에 추가할 수 있습니다.
    // purchaseManager?.launchPurchaseFlow(context, itemId, listener, bundle)
}

// 콜백 인터페이스 예시
interface PurchaseListener {
    fun onPurchaseSuccess(purchaseInfo: PurchaseInfo)
    fun onPurchaseFailure(error:

In [ ]:
# 검색 결과 분석
def analyze_search_results(question: str, top_k: int = 5):
    """검색 결과를 분석하여 상세 정보를 제공합니다."""
    print(f"🔍 질문: {question}")
    print(f"📊 상위 {top_k}개 검색 결과 분석\\n")
    
    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    
    for i, doc in enumerate(docs[:top_k], 1):
        print(f"📄 결과 {i}:")
        print(f"   타입: {doc.metadata.get('type', 'unknown')}")
        print(f"   소스: {doc.metadata.get('source', 'unknown')}")
        print(f"   키워드: {doc.metadata.get('keywords', [])}")
        print(f"   코드 포함: {doc.metadata.get('has_code', False)}")
        print(f"   함수 포함: {doc.metadata.get('has_function', False)}")
        print(f"   내용: {doc.page_content[:150]}...")
        print("   " + "-" * 50)
    
    print(f"\\n✅ 총 {len(docs)}개의 관련 문서를 찾았습니다.")

# 검색 결과 분석 테스트
analyze_search_results("launchPurchaseFlow", top_k=3)
print("\\n" + "=" * 80)
analyze_search_results("PurchaseClient", top_k=3)